# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Anderson Jiménez Torres <br>
Url: https://github.com/97anderson/AlgoritmosDeOptimizacion.git<br>
Google Colab: https://drive.google.com/file/d/1b8i_ZNGB-RAkDd0UMimxHxzlYfddFPr3/view?usp=sharing  <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las 
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de 
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de 
grabación independientemente del número de tomas que se graben. No es posible grabar más 
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los 
servicios de los actores de doblaje sea el menor posible. Los datos son:
Número de actores: 10
Número de tomas : 30
Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario








                                        

# Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

#### ¿Como represento el espacio de soluciones?
El espacio de soluciones se representa mediante la asignación de actores a escenas y de escenas o tomas a dias. Esto se realiza mediante la matriz 
escenas_x_actor, donde cada fila representa una toma y cada columna representa un actor. Un valor de 1 en una posición indica que el actor 
participa en esa toma, mientras que un valor de 0 indica lo contrario.


#### ¿Cual es la función objetivo?
Maximizar la cantidad de tomas que se pueden realiza con tal de reducir costos, creando un horario o planeación que no haga que un actor vaya mas de
lo necesario y de esa manera pagando lo menos posible.

En el algoritmo desarrollado se realiza una busqueda por actores que menos participan en el doble y se crea una matriz de menos a mas, se acomoda los 
actores que menos participan a los que mas participan. Esto con el fin de no hacer venir mas días de los necesarios al mismo actor, con tal de reducir costos en el doblaje.<br>
A continuación se comienza a desarrollar el código de lo que se explico anteriormente.


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Se crea dataset a partir de archivo csv suministrado
datosDoblaje = pd.read_csv('Datos_problema_doblaje.csv', delimiter=',')
print(datosDoblaje)

   Unnamed: 0  Actor  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0        Toma    1.0         2.0         3.0         4.0         5.0   
1           1    1.0         1.0         1.0         1.0         1.0   
2           2    0.0         0.0         1.0         1.0         1.0   
3           3    0.0         1.0         0.0         0.0         1.0   
4           4    1.0         1.0         0.0         0.0         0.0   
5           5    0.0         1.0         0.0         1.0         0.0   
6           6    1.0         1.0         0.0         1.0         1.0   
7           7    1.0         1.0         0.0         1.0         1.0   
8           8    1.0         1.0         0.0         0.0         0.0   
9           9    1.0         1.0         0.0         1.0         0.0   
10         10    1.0         1.0         0.0         0.0         0.0   
11         11    1.0         1.0         1.0         0.0         1.0   
12         12    1.0         1.0         1.0         1.0        

In [6]:
# DECLARACION DE VARIABLES GLOBALES

cantidad_escenas = datosDoblaje.iloc[1:-2, 0].copy().values.astype(int)  # numero total de escenas, sacadas del archivo csv
num_actores = len(datosDoblaje.columns) - 3  # Cantidad total de actores de la pelicula, sacando columnas que no sirven
contadores_por_actor = [0] * num_actores  # Se inicializa array con tamaño total de actores para sumar las escenas diarias, no > 6
escenas_x_actor = datosDoblaje.iloc[1:-2, 1:-2].copy().values.astype(int)  # Matriz con escenas en que participa cada actor
dias_grabacion = 1  # Se inicializa dia de grabacion en 1
sesiones_por_dia = {}  # diccionario que tendra las escenas por dias de grabación
total_actores_escena = datosDoblaje.iloc[1:-2, -1].copy().values.astype(int)  # Total de actores por escenas 

total_escenas_actores = datosDoblaje.iloc[[0, -1], 1:num_actores + 1] # Se consigue cantidad de escenas de cada actor
actores_con_escenas = total_escenas_actores.values.astype(int)
array_total_tomas_actor = actores_con_escenas[1]
indice_orden = np.argsort(actores_con_escenas[1])  # Orden de menor a mayor de fila final
actores_con_escenas = actores_con_escenas[:, indice_orden]  # Aplicamos el mismo orden a la primera fila

contador_escenas = len(cantidad_escenas)
horario_doblaje = {}
tomas_programadas = []  # Lista para almacenar las tomas ya programadas
actores_participan_tomas = 0 # variable que identifica los actores que participan en una toma que se va a realizar.

#### ¿Como implemento las restricciones?
Las restircciones como el enunciado inicial lo dice son basicamente que un actor tenga un maximo de 6 tomas por día y reducir las ida de cada actor al minimo posible con tal de que el actor vaya solo lo necesario, si en el mismo dia que va se pueden hacer las tomas en las que va a participar mucho mejor.<br>

Por ende se realiza un recorrido de los actores del que menos participa al que mas y se van revisando las escenas en las que participa, se descuenta la escena del maximo de tomas que hace el actor indicado por la ultima fila del archivo csv, teniendo en cuenta o llevando un contador de las 6 que puede hacer en el día, el código es el siguiente: 

In [7]:
# Se valida actor con escenas minimas a empezar a validar tomando variable actores_con_escenas fila 2
for posicion in range(num_actores):
    actor = actores_con_escenas[0, posicion]  # Se valida actor que menos participa a max
    numero_escenas = 0
    
    while array_total_tomas_actor[actor - 1] > 0:
        
        for escena in range(len(cantidad_escenas)):
            numero_escenas += 1

            if escenas_x_actor[escena, actor - 1] == 1 and contador_escenas > 0 and escena not in tomas_programadas:
                participantes = [i for i in range(num_actores) if escenas_x_actor[escena, i] == 1]
                
                for i in participantes:
                    contadores_por_actor[i] += 1
                    if contadores_por_actor[i] < 6:
                        actores_participan_tomas += 1
    
                if all(contadores_por_actor[i] < 6 for i in participantes) and actores_participan_tomas == total_actores_escena[escena]:
                    sesiones_por_dia.setdefault(dias_grabacion, []).append(escena)
                    contador_escenas -= 1
                    tomas_programadas.append(escena)
                    
                    for i in participantes:
                        array_total_tomas_actor[i] -= 1
                
                actores_participan_tomas = 0
                
                if any(contadores_por_actor[i] >= 6 for i in participantes) and numero_escenas >= len(cantidad_escenas):
                    dias_grabacion += 1
                    contadores_por_actor = [0] * num_actores
                    numero_escenas = 0
            

Finalmente después de hacer el recorrido anterior e ir revisando actor y tomas o escenas de cada actor y agregarlas a un dia determinado, hemos hallado una configuración optima para el horario en el que podemos hacer ir a los actores al doblaje de la pelicula, el resultado optenido es el siguiente:

In [8]:
# Imprimir el horario de doblaje por día
for dia, sesiones in sesiones_por_dia.items():
    if sesiones:
        print(f"Día {dia}: {len(sesiones)} sesiones")
        for sesion in sesiones:
            participantes = [i + 1 for i, valor in enumerate(escenas_x_actor[sesion]) if valor == 1]
            print(f"    Escena {sesion + 1}: Participantes {participantes}")
    else:
        print(f"Día {dia}: 0 sesiones")

Día 1: 8 sesiones
    Escena 10: Participantes [1, 2, 6, 9]
    Escena 26: Participantes [1, 3, 5, 9]
    Escena 16: Participantes [4, 10]
    Escena 25: Participantes [1, 2, 4, 10]
    Escena 3: Participantes [2, 5, 7]
    Escena 4: Participantes [1, 2, 7, 8]
    Escena 15: Participantes [1, 2, 7]
    Escena 21: Participantes [6, 8]
Día 2: 7 sesiones
    Escena 11: Participantes [1, 2, 3, 5, 8]
    Escena 5: Participantes [2, 4, 8]
    Escena 8: Participantes [1, 2, 6]
    Escena 12: Participantes [1, 2, 3, 4, 6]
    Escena 14: Participantes [1, 3, 6]
    Escena 18: Participantes [3, 6]
    Escena 24: Participantes [3, 6]
Día 3: 5 sesiones
    Escena 29: Participantes [1, 5, 6]
    Escena 1: Participantes [1, 2, 3, 4, 5]
    Escena 2: Participantes [3, 4, 5]
    Escena 6: Participantes [1, 2, 4, 5]
    Escena 7: Participantes [1, 2, 4, 5]
Día 4: 6 sesiones
    Escena 20: Participantes [1, 3, 4, 5]
    Escena 27: Participantes [4, 5]
    Escena 13: Participantes [1, 4, 5]
    Escena 17

# Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

El problema anterior resulto por el código descrito tiene una complejidad que depende de varios factores, entre ellos el número de tomas o escenas, 
el número de actores y la distribución de la participación de los actores en las tomas.

#### Orden de coejidad
- El bucle principal itera sobre todos los actores y luego sobre todas las escenas, lo que da como resultado una complejidad de tiempo de O(num_actores * num_escenas).
- Dentro de este bucle, hay operaciones adicionales como verificar la disponibilidad de los actores para una escena específica, lo que agrega cierta complejidad adicional.
- La asignación de escenas a actores se realiza de manera voraz, lo que puede llevar a una solución subóptima pero rápida. Por lo tanto, la complejidad real puede variar según la distribución de las escenas y la participación de los actores.dad

#### Contabilizar el espacio de soluciones
- El espacio de soluciones puede contabilizarse examinando todas las combinaciones posibles de asignación de actores a escenas.
- Dado que para cada escena hay un número finito de actores posibles y el número de escenas es finito, el espacio de soluciones será la cantidad total de combinaciones mosibles de asignaciones de actores a escenas.
- Esta cantidad puede ser enorme, especialmente en películas con muchas escenas y actores. Por lo tanto, el enfoque voraz utilizado en el código puede ser beneficioso para reducir el tiempo de búsqueda de una solución factible, aunque puede no ser óptimo en términos de encontrar la mejor solución posible.

# Diseño
- ¿Que técnica utilizo? ¿Por qué?

El código proporcionado utiliza una técnica de programación voraz (greedy) para resolver el problema de asignar escenas de doblaje a actores. 
Se elige esta técnica por varias razones:

- Eficiencia en tiempo de ejecución: Los algoritmos voraces suelen ser más rápidos que otros enfoques, ya que toman decisiones locales en cada paso sin tener en cuenta el futuro. Esto los hace adecuados para problemas donde se necesita una solución rápida.
- Facilidad de implementación: La implementación de algoritmos voraces tiende a ser más simple y directa en comparación con otros enfoques como la programación dinámica o los algoritmos genéticos.
- Adecuación al problema: En este caso específico, el problema de asignar escenas de doblaje a actores puede abordarse de manera voraz, ya que en cada paso se selecciona la próxima escena disponible y se asignan actores a ella hasta que se cumplan todas las restricciones. No hay una necesidad inmediata de considerar todas las posibles combinaciones de asignaciones de actores a escenas, lo que hace que un enfoque voraz sea adecuado.